In [18]:
import pandas as pd
import numpy as np
import joblib
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, rdChemReactions
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# --- Load model and scaler ---
scaler = joblib.load('scaler.pkl')
model = joblib.load('mlp_classifier_model.pkl')

# --- Load templates dataframe ---
file_path = "/Users/giuliogarotti/Documents/GitHub/Projet_chem/uspto50/uspto50/reaction_templates_50k_train.csv"
templates_df = pd.read_csv(file_path, sep='\t')

# --- Functions ---
def smiles_to_fingerprint(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string")
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

def apply_template(template_smarts, smiles_input):
    mol = Chem.MolFromSmiles(smiles_input)
    if mol is None:
        return []
    try:
        rxn = rdChemReactions.ReactionFromSmarts(template_smarts)
        products = rxn.RunReactants((mol,))
        product_smiles = []
        for prod_set in products:
            prod_list = [Chem.MolToSmiles(p) for p in prod_set if p is not None]
            product_smiles.append(prod_list)
        return product_smiles
    except:
        return []

def predict_topk_templates(smiles_input, topk=50):
    fingerprint = smiles_to_fingerprint(smiles_input)
    fingerprint = fingerprint.reshape(1, -1)
    fingerprint_scaled = scaler.transform(fingerprint)

    # Get probabilities
    probs = model.predict_proba(fingerprint_scaled)[0]

    # Top-k
    topk_indices = np.argsort(probs)[::-1][:topk]
    topk_template_hashes = model.classes_[topk_indices]
    topk_probs = probs[topk_indices]

    predictions = []
    for template_hash, prob in zip(topk_template_hashes, topk_probs):
        row = templates_df[templates_df['TemplateHash'] == template_hash]
        if not row.empty:
            retro_template = row.iloc[0]['RetroTemplate']
            predictions.append((template_hash, retro_template, prob))
        else:
            predictions.append((template_hash, None, prob))
    return predictions


# --- Main upgraded code ---
if __name__ == "__main__":
    smiles_input = input("Enter a SMILES string of the product: ")
    try:
        topk_predictions = predict_topk_templates(smiles_input, topk=50)

        successful_predictions = []

        for rank, (template_hash, retro_template, prob) in enumerate(topk_predictions, start=1):
            if retro_template is None:
                continue

            predicted_products = apply_template(retro_template, smiles_input)

            if predicted_products:
                successful_predictions.append((template_hash, retro_template, prob, predicted_products))

        # Display successful predictions
        if successful_predictions:
            print("\n✅ Successful Predictions:")
            for idx, (template_hash, retro_template, prob, products) in enumerate(successful_predictions, start=1):
                print(f"\n🔹 Success {idx}")
                print(f"TemplateHash: {template_hash}")
                print(f"Model Probability: {prob*100:.2f}%")
                print(f"Template SMARTS: {retro_template}")
                print(f"Predicted Reactants:")
                for prod_set in products:
                    print(f"   {prod_set}")
        else:
            print("❗ No valid templates generated products for this molecule.")

    except Exception as e:
        print(f"Error: {e}")





[11:47:50] DEPRECATION WARNING: please use MorganGenerator



✅ Successful Predictions:

🔹 Success 1
TemplateHash: 5168419636a1f679b83d61fc353e455085e43b6460777dfdc0e4fbcd717e2b61
Model Probability: 10.66%
Template SMARTS: [OH;D1;+0:1]-[CH2;D2;+0:2]-[c:3]>>[O;H0;D1;+0:1]=[CH;D2;+0:2]-[c:3]
Predicted Reactants:
   ['O=Cc1ccccc1']

🔹 Success 2
TemplateHash: 58461fa3005f8f4798e5854e06697f99dd7eeecb9f907674dcf3bc28f006ca29
Model Probability: 6.91%
Template SMARTS: [O;D1;H1:2]-[CH2;D2;+0:1]-[c:3]>>O=[C;H0;D3;+0:1](-[O;D1;H1:2])-[c:3]
Predicted Reactants:
   ['O=C(O)c1ccccc1']

🔹 Success 3
TemplateHash: bd784f060866bc639d42cf1a699c65c68907bb6d79b1f1f882b00844875f5340
Model Probability: 2.10%
Template SMARTS: [OH;D1;+0:2]-[CH2;D2;+0:1]-[c:3]1:[c:4]:[c:5]:[c:6]:[c:7]:[c:8]:1>>C-O-[C;H0;D3;+0:1](=[O;H0;D1;+0:2])-[c:3]1:[c:4]:[c:5]:[c:6]:[c:7]:[c:8]:1
Predicted Reactants:
   ['COC(=O)c1ccccc1']
   ['COC(=O)c1ccccc1']

🔹 Success 4
TemplateHash: 5a6df5201d3deedc0c8a2e95760ac64e7b20bed15455d053a10230fddcd650d2
Model Probability: 0.29%
Template SMARTS: [C:2]-

In [ ]:
#Test molecules 

#Acetophenone: 
CC(=O)c1ccccc1
#Benzyl alcohol:
CC(C1=CC=CC=C1)O

In [20]:
# --- Visualization code ---
from rdkit import Chem
from rdkit.Chem import Draw

# List of SMILES strings
smiles_list = ['CC(C)(C)[Si](C)(C)OCc1ccccc1']

# Convert SMILES to RDKit molecule objects
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

# Draw the molecules
img = Draw.MolsToImage(mols, molsPerRow=len(mols))

# Show the image
img.show()